In [ ]:
import sqlite3
import csv

def run_query(sql_query, db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    cursor.execute(sql_query)

    rows = cursor.fetchall()
    
    print(f"Number of the rows : { len(rows) }")

    for row in rows[:3]: print(row)

    conn.close()


def create_table(db_name, tables):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    for table_name, schema in tables.items():
        cursor.execute(f"CREATE TABLE IF NOT EXISTS { table_name } ({ schema })")

    conn.commit()
    conn.close()


def populate_table_from_csv(db_name, table_name, csv_file):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    with open(csv_file, 'r', newline='', encoding='utf-8') as csvfile:
        lines = csvfile.readlines()
        for line in lines[1:]:
            values = line.strip().split(',')
            print(values)
            cursor.execute(f"INSERT INTO { table_name } VALUES ({','.join( ['?'] * len(values) )})", values)

    conn.commit()
    conn.close()

In [ ]:
# Setup

Tables = {
    'BazaarApps': 'Name TEXT PRIMARY KEY, Developer TEXT, Installs INTEGER, Category_FA TEXT, Size TEXT, Rating INTEGER, Votes INTEGER, Five_Stars INTEGER, One_Stars INTEGER', 

    'MyketApps': 'Name TEXT PRIMARY KEY, Installs INTEGER, Rating INTEGER, Votes INTEGER, Category_FA TEXT, Category_EN TEXT',
}

create_table('IrAppStores.db', Tables)

populate_table_from_csv('IrAppStores.db', 'BazaarApps', 'DataSets/BazaarApps.csv')

populate_table_from_csv('IrAppStores.db', 'MyketApps', 'DataSets/MyketApps.csv')